In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-quad-preprocessed/preprocessed_test_dataset.csv
/kaggle/input/quora-quad-preprocessed/preprocessed_train_dataset.csv


In [2]:
%pip install transformers datasets evaluate torch nltk rouge_score
!pip install sacrebleu

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6705e6b6e083f8a42903af14a693cba5c12bde04be2c7473687552808baaa029
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.9 MB/s eta 0:00:00


## Original Dataset

In [3]:
# from datasets import Dataset, DatasetDict
# from datasets import load_dataset
# dataset = load_dataset("toughdata/quora-question-answer-dataset")
# dataset = dataset["train"].train_test_split(test_size=0.2)

## Preprocessed Dataset

In [4]:
import pandas as pd

# Load preprocessed datasets
df_test_preprocessed = pd.read_csv('/kaggle/input/quora-quad-preprocessed/preprocessed_test_dataset.csv')
df_train_preprocessed = pd.read_csv('/kaggle/input/quora-quad-preprocessed/preprocessed_train_dataset.csv')

from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(df_train_preprocessed)
test_dataset = Dataset.from_pandas(df_test_preprocessed)
dataset_preprocessed= DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


print("Datasets converted back to original format for model training.")

Datasets converted back to original format for model training.


In [5]:
from datasets import Dataset, DatasetDict

# Ensure 'question' and 'answer' columns are of type string
df_train_preprocessed['question'] = df_train_preprocessed['question'].astype(str)
df_train_preprocessed['answer'] = df_train_preprocessed['answer'].astype(str)

df_test_preprocessed['question'] = df_test_preprocessed['question'].astype(str)
df_test_preprocessed['answer'] = df_test_preprocessed['answer'].astype(str)

train_dataset = Dataset.from_pandas(df_train_preprocessed)
test_dataset = Dataset.from_pandas(df_test_preprocessed)

dataset_preprocessed = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

print(dataset_preprocessed)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 44258
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 11187
    })
})


In [ ]:
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch
from sklearn.metrics import precision_recall_fscore_support

# Ensure GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the tokenizer and model for DistilGPT-2
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("distilgpt2").to(device)

# Load the dataset
# dataset = load_dataset("quora")

# Define fraction of the dataset to use
fraction = 0.0005  # Use 0.05% of the dataset

# Create smaller subsets of the dataset
train_size = int(len(dataset["train"]) * fraction)
test_size = int(len(dataset["test"]) * fraction)

# Shuffle and select the subset
train_subset = dataset["train"].shuffle(seed=42).select(range(train_size))
test_subset = dataset["test"].shuffle(seed=42).select(range(test_size))

prefix = "answer the question: "

def preprocess_function(examples):
    """Concatenate question and answer with a prefix, and tokenize"""
    inputs = [prefix + doc for doc in examples["question"]]
    outputs = examples["answer"]
    model_inputs = tokenizer(inputs, max_length=64, truncation=True, padding="max_length")
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(outputs, max_length=64, truncation=True, padding="max_length")

    # Concatenate input and output for causal language modeling
    model_inputs["input_ids"] = [input_ids + label_ids for input_ids, label_ids in zip(model_inputs["input_ids"], labels["input_ids"])]
    model_inputs["attention_mask"] = [attn_mask + [1] * len(label_ids) for attn_mask, label_ids in zip(model_inputs["attention_mask"], labels["input_ids"])]

    # Add padding token where necessary
    max_length = max(len(x) for x in model_inputs["input_ids"])
    model_inputs["input_ids"] = [x + [tokenizer.pad_token_id] * (max_length - len(x)) for x in model_inputs["input_ids"]]
    model_inputs["attention_mask"] = [x + [0] * (max_length - len(x)) for x in model_inputs["attention_mask"]]

    # Shift the labels so the decoder learns to predict the next token
    model_inputs["labels"] = [[-100] * len(input_ids) + label_ids for input_ids, label_ids in zip(model_inputs["input_ids"], labels["input_ids"])]

    return model_inputs

# Tokenize the subset datasets
tokenized_train_dataset = train_subset.map(preprocess_function, batched=True, remove_columns=["question", "answer"])
tokenized_test_dataset = test_subset.map(preprocess_function, batched=True, remove_columns=["question", "answer"])

# Load the data collator for GPT-2
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set up ROUGE, BLEU, Precision, Recall, and F1 score for evaluation
nltk.download("punkt", quiet=True)
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Convert predictions and labels to lists and flatten
    preds = preds.tolist()
    labels = labels.tolist()

    # Debugging information
    print(f"First 5 preds: {preds[:5]}")
    print(f"First 5 labels: {labels[:5]}")

    # Flatten nested lists
    preds = [item for sublist in preds for item in sublist if item is not None]
    labels = [item for sublist in labels for item in sublist if item is not None]

    # Decode preds and labels
    try:
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    except TypeError as e:
        print(f"Error during decoding: {e}")
        print(f"Preds: {preds}")
        print(f"Labels: {labels}")
        raise

    # ROUGE expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Compute ROUGE score
    rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Compute BLEU score
    decoded_preds_for_bleu = [" ".join(pred.split()) for pred in decoded_preds]
    decoded_labels_for_bleu = [[" ".join(label.split())] for label in decoded_labels]
    bleu_result = bleu_metric.compute(predictions=decoded_preds_for_bleu, references=decoded_labels_for_bleu)

    # Compute Precision, Recall, and F1 on sentence level
    true_labels = [label.split() for label in decoded_labels]
    pred_labels = [pred.split() for pred in decoded_preds]

    # Flatten the lists
    true_labels_flat = [token for sublist in true_labels for token in sublist]
    pred_labels_flat = [token for sublist in pred_labels for token in sublist]
    
    # Ensure the lengths match by truncating to the minimum length
    min_length = min(len(true_labels_flat), len(pred_labels_flat))
    true_labels_flat = true_labels_flat[:min_length]
    pred_labels_flat = pred_labels_flat[:min_length]
    
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels_flat, pred_labels_flat, average='weighted', zero_division=0)

    # Combine metrics
    combined_result = {
        **rouge_result,
        "bleu": bleu_result["score"],
        "precision": precision,
        "recall": recall,
        "f1": f1
    }
    return combined_result

# Set up training arguments with logging configuration
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,  # Increase or decrease as needed
    learning_rate=5e-5,  # Adjust learning rate
    per_device_train_batch_size=1,  # Further reduced batch size
    per_device_eval_batch_size=1,  # Further reduced batch size
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch size
    fp16=True,  # Use mixed precision training
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,  # Reduce number of epochs
    report_to="none",  # Use "tensorboard" if needed
)

# Clear the CUDA cache
torch.cuda.empty_cache()

# Set up trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


In [ ]:
trainer.save_model('./results/final_model')